<a href="https://colab.research.google.com/github/nrana00/Assignment-Week-9/blob/main/Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!java --version
!python --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Python 3.9.16


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=7a4440148d6712d2bb24848463cc203255de777c6452680034e3c95c66eb395f
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
# Check spark session
print(spark)

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [15]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=3371fb6cbcfec1581ae47cbaa569aa7242e448a7e3a69b576c800148dee55daf
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [20]:
import wget

In [21]:
file = wget.download("https://raw.githubusercontent.com/apache/spark/master/data/mllib/als/sample_movielens_ratings.txt")

In [41]:
lines = spark.read.text("/content/sample_movielens_ratings.txt").rdd 
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

In [53]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=20, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [54]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.0258667144534424


In [37]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{22, 4.5668893},...|
|    10|[{2, 4.109646}, {...|
|     0|[{92, 3.837613}, ...|
|     1|[{46, 5.087489}, ...|
|    21|[{53, 5.0437827},...|
|    11|[{27, 5.274435}, ...|
|    12|[{46, 7.070434}, ...|
|    22|[{46, 6.5465045},...|
|     2|[{93, 5.116105}, ...|
|    13|[{93, 3.8023236},...|
|     3|[{51, 4.7560487},...|
|    23|[{90, 5.8201103},...|
|     4|[{46, 4.4920945},...|
|    24|[{76, 5.9640956},...|
|    14|[{85, 5.1566653},...|
|     5|[{46, 6.103309}, ...|
|    15|[{46, 4.467009}, ...|
|    25|[{47, 4.0174255},...|
|    26|[{75, 6.2839193},...|
|     6|[{25, 5.1459274},...|
+------+--------------------+
only showing top 20 rows



In [42]:
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{17, 4.8482265},...|
|     40|[{2, 4.4584074}, ...|
|     10|[{17, 3.840151}, ...|
|     50|[{11, 4.0581503},...|
|     80|[{26, 4.4764695},...|
|     70|[{21, 3.7892954},...|
|     60|[{12, 3.0825105},...|
|     90|[{23, 5.8201103},...|
|     30|[{11, 5.0238}, {2...|
|      0|[{28, 3.0133045},...|
|     31|[{1, 3.5833948}, ...|
|     81|[{28, 4.589747}, ...|
|     91|[{8, 3.853937}, {...|
|      1|[{16, 4.39842}, {...|
|     41|[{4, 3.9864016}, ...|
|     61|[{6, 2.5738652}, ...|
|     51|[{26, 5.1646247},...|
|     21|[{28, 3.0664046},...|
|     11|[{18, 4.016637}, ...|
|     71|[{9, 3.9970198}, ...|
+-------+--------------------+
only showing top 20 rows



In [43]:
# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{75, 6.2839193},...|
|    19|[{51, 4.6269445},...|
|    29|[{46, 4.7928276},...|
+------+--------------------+



In [44]:
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     65|[{12, 3.1439056},...|
|     26|[{12, 4.1143427},...|
|     29|[{26, 5.3168445},...|
+-------+--------------------+



The model with the lower RMSE is generally considered to perform better.
The model that performs better according to the RMSE is the model with maxIter=20, regParam=0.1.

maxIter=5, regParam=0.1 the RMSE is = 1.0536207567527969

maxIter=10, regParam=0.1 the RMSE is = 1.0276701524655754

maxIter=20, regParam=0.1 the RMSE is = 1.0258667144534424

maxIter=10, regParam=0.5 the RMSE is = 1.319668327611824

maxIter=20, regParam=1.0 the RMSE is = 1.5999452338310785